In [2]:
# model.py
# ---------------------------
# Chargement du dataset + préparation NLP + TF-IDF
# Utilisé par recommender.py et app.py
# ---------------------------

import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS

# ============================================================
# 1) Charger le dataset
# ============================================================

# IMPORTANT : adapte ce chemin selon l'endroit où se trouve ton fichier
df = pd.read_csv("C:\\Users\\pc\\Desktop\\course-recommendation\\data\\courses.csv")

# Supprimer lignes sans titre (sécurité)
df = df.dropna(subset=["title"]).copy()

print("\n=================== ÉTAPE 1 : Dataset chargé ===================")
print("Taille du dataset (lignes, colonnes) :", df.shape)
print("\nAperçu des 5 premières lignes :")
print(df[["id", "title"]].head())
print("\nTypes des colonnes :")
print(df.dtypes)
print("================================================================\n")

# Vérification / conversion de l'ID
if df["id"].dtype != "int64" and df["id"].dtype != "int32":
    df["id"] = df["id"].astype(int)

print("=================== ÉTAPE 2 : Vérification de la colonne ID ===================")
print("Type de la colonne id :", df["id"].dtype)
print("Quelques id :", df["id"].head().tolist())
print("================================================================\n")


# ============================================================
# 2) Fonction de nettoyage (NLP simple)
# ============================================================

def clean_text(text: str) -> str:
    """
    Nettoyage simple du titre :
    - minuscules
    - suppression de la ponctuation / chiffres
    - suppression des stopwords anglais
    """
    text = text.lower()
    text = re.sub(r"[^a-zA-Z\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()

    tokens = text.split()
    tokens = [t for t in tokens if t not in ENGLISH_STOP_WORDS]

    return " ".join(tokens)


# Appliquer le nettoyage
df["clean_title"] = df["title"].apply(clean_text)

print("=================== ÉTAPE 3 : Nettoyage NLP ===================")
print("Aperçu avant / après nettoyage :")
print(df[["title", "clean_title"]].head(10))
print("================================================================\n")


# ============================================================
# 3) Vectorisation TF-IDF
# ============================================================

vectorizer = TfidfVectorizer()

# Entraîner TF-IDF sur le texte nettoyé
tfidf_matrix = vectorizer.fit_transform(df["clean_title"])

print("=================== ÉTAPE 4 : TF-IDF ===================")
print("Taille de la matrice TF-IDF :", tfidf_matrix.shape)
print("Nombre de cours      :", tfidf_matrix.shape[0])
print("Taille du vocabulaire:", tfidf_matrix.shape[1])

feature_names = vectorizer.get_feature_names_out()
print("\nExemples de mots du vocabulaire TF-IDF :")
print(feature_names[:30])
print("================================================================\n")


# ============================================================
# 4) Vectorisation d'un texte externe (optionnel)
# ============================================================

def vectorize_text(text: str):
    clean = clean_text(text)
    return vectorizer.transform([clean])


# ============================================================
# Exposé aux autres fichiers :
# - df : DataFrame des cours
# - tfidf_matrix : matrice TF-IDF
# - vectorizer : modèle TF-IDF
# - clean_text(), vectorize_text() : utilitaires NLP
# ============================================================

print("=================== MODEL.PY CHARGÉ AVEC SUCCÈS ===================\n")



=================== ÉTAPE 1 : Dataset chargé ===================
Taille du dataset (lignes, colonnes) : (13608, 20)

Aperçu des 5 premières lignes :
        id                                              title
0   762616  The Complete SQL Bootcamp 2020: Go from Zero t...
1   937678  Tableau 2020 A-Z: Hands-On Tableau Training fo...
2  1361790             PMP Exam Prep Seminar -  PMBOK Guide 6
3   648826         The Complete Financial Analyst Course 2020
4   637930  An Entire MBA in 1 Course:Award Winning Busine...

Types des colonnes :
id                                int64
title                            object
url                              object
is_paid                            bool
num_subscribers                   int64
avg_rating                      float64
avg_rating_recent               float64
rating                          float64
num_reviews                       int64
is_wishlisted                      bool
num_published_lectures            int64
num_published_pr

In [5]:
import os
os.makedirs("model", exist_ok=True)

In [6]:
import joblib

print("➡ Sauvegarde des fichiers du modèle...")

joblib.dump(df, "model/courses_df.pkl")
joblib.dump(vectorizer, "model/tfidf_vectorizer.pkl")
joblib.dump(tfidf_matrix, "model/tfidf_matrix.pkl")

print("✅ Sauvegarde terminée : fichiers prêts pour la recommandation !")


➡ Sauvegarde des fichiers du modèle...
✅ Sauvegarde terminée : fichiers prêts pour la recommandation !
